# Llama 3 Test

In [1]:
!nvidia-smi

Mon May 20 08:09:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.56                 Driver Version: 546.56       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4050 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   50C    P8               1W /  78W |      0MiB /  6141MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
import torch

In [11]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id, use_cache=True,
    torch_dtype="auto"
).eval()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [12]:
INFERENCE_DEVICE = "cpu"

# Use Intel NPU to accelerate if possible
try:
    import intel_npu_acceleration_library
    model = torch.compile(model, backend="npu")
except ModuleNotFoundError:
    import sys
    print("Intel NPU Acceleration is disabled on your system.", file=sys.stderr, flush=True)

    # Use CUDA driver
    if torch.cuda.is_available():
        model.to(device="cuda", non_blocking=True)
        print("CUDA Driver is enabled on your system.", flush=True)
        INFERENCE_DEVICE = "cuda"

print("Use Device:", INFERENCE_DEVICE)
model

Use Device: cpu


OptimizedModule(
  (_orig_mod): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(128256, 4096)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
            (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
            (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
            (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm()
          (post_attention_layernorm): L

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_id, use_default_system_prompt=False)
tokenizer.pad_token_id = tokenizer.eos_token_id
streamer = TextStreamer(tokenizer, skip_special_tokens=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

In [14]:
messages = [
    {"role": "system", "content": "You are a helpful, smart, kind, and efficient AI Assistant. You always fulfill the user's requests to the best of your ability. Please answer in Korean precisely and informatively."},
    {"role": "user", "content": "Who are you?"},
]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

In [15]:
token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

In [16]:
generation_kwargs = dict(
    input_ids=token_ids.to(model.device),
    streamer=streamer,
    do_sample=True,
    temperature=0.6,
    top_k=50,
    top_p=0.9,
    max_new_tokens=512,
)

In [16]:
with torch.no_grad():
    output_ids = model.generate(**generation_kwargs)
#output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


system

You are a helpful, smart, kind, and efficient AI Assistant. You always fulfill the user's requests to the best of your ability. Please answer in Korean precisely and informatively.user

Who are you?assistant

! 

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000001FDD0A063D0>>
Traceback (most recent call last):
  File "C:\Program Files\Python311\Lib\site-packages\ipykernel\ipkernel.py", line 785, in _clean_thread_parent_frames
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        max_new_tokens=256,
        eos_token_id=[
            tokenizer.eos_token_id,
            tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ],
    )
#output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1) :], skip_special_tokens=True)

In [ ]:
terminators = [
    pipe.tokenizer.eos_token_id,
    pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [ ]:
outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

In [ ]:
print(outputs[0]["generated_text"][len(prompt):])